# Progressive GAN

In [403]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader

# Data

In [404]:
def transform_in_size(size):
    return Compose([Resize((size, size)), ToTensor()])

def get_dataloader(size):
    dataset = CIFAR10('~/pytorch', train=True, download=True, transform=transform_in_size(size))
    return DataLoader(dataset, batch_size=32)

# Generator
* TODO: Change UpsamplingBilinear to interpolate()
* TODO: Move to inner class

In [405]:
class GeneratorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        
    def forward(self, x):
        _, x = self.forward_with_img(x)
        
        return x
    
    def forward_with_img(self, x):
        img = self.upsample(x)
        x = F.leaky_relu(self.conv1(img))
        x = F.leaky_relu(self.conv2(x))
        
        return img, x

In [406]:
class Generator(nn.Module):
    def __init__(self, channels=512):
        super().__init__()
        self.channels = channels
        self.trained_blocks = nn.ModuleList()
        self.new_block = self.create_initial_layer(channels)
        self.rgb = self.to_rgb(channels)
        
    def to_rgb(self, in_channels):
        return nn.Conv2d(in_channels, 3, 1, padding=0)
    
    def create_initial_layer(self, channels):
        module = nn.Sequential(
            nn.Conv2d(channels, channels, 4, padding=3),
            nn.LeakyReLU(),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.LeakyReLU())
        
        return module
    
    def append_layer(self, channels):
        self.trained_blocks.append(self.new_block)
        self.new_block = GeneratorBlock(self.channels, channels)
        self.rgb = self.to_rgb(channels)
        self.channels = channels
        
    def forward(self, x, alpha=1.0):
        for block in self.trained_blocks:
            x = block(x)
            
        if alpha < 1.0:
            img , x = self.new_block.forward_with_img(x)
            img = self.rgb(img)
            x = self.rgb(x)
            
            x = img * (1 - alpha) + img * alpha
        else:
            x = self.new_block(x)
            x = self.rgb(x)
        
        return x

# Disciminator

In [407]:
class DiscriminatorBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.downsample = nn.UpsamplingBilinear2d(scale_factor=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.downsample(x)
        
        return x

In [408]:
class Discriminator(nn.Module):
    def __init__(self, channels=512):
        super().__init__()
        self.channels = channels
        self.trained_layers = nn.ModuleList()
        self.new_layer = self.create_initial_layer(channels)
        self.downsample = nn.UpsamplingBilinear2d(scale_factor=0.5)
        self.rgb = self.from_rgb(channels)
        
    def create_initial_layer(self, channels):
        module = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.Conv2d(channels, channels, 4, padding=0),
            nn.Flatten(),
            nn.Linear(channels, 1))
        
        return module
    
    def prepend_layer(self, channels):
        self.trained_layers.insert(0, self.new_layer)
        self.new_layer = DiscriminatorBlock(channels, self.channels)
        self.rgb = self.from_rgb(channels)
        
    def from_rgb(self, channels):
        return nn.Conv2d(3, channels, 1, padding=0)
        
    def forward(self, x, alpha=1.0):
        if alpha < 1.0:
            skip = self.downsample(x)
            skip = self.rgb(skip)

            x = self.rgb(x)
            x = self.new_layer(x)
            x = skip * (1-alpha) + x * alpha
        else:
            x = self.rgb(x)
            x = self.new_layer(x)
        
        for layer in self.trained_layers:
            x = layer(x)
        
        return x

In [409]:
def train_batch():
    z = torch.randn(32, 512, 1, 1)
    gen_img = g(z)
    print(f"Image size: {gen_img.shape}")
    out = d(gen_img)
    
    print(out.shape)

In [410]:
g = Generator()
d = Discriminator()

In [417]:
def train_epoch():
    d_criterion = nn.BCEWithLogitsLoss()
    d_optim = Adam(d.parameters())
    g_optim = Adam(g.parameters())
    
    dl = get_dataloader(4)
    real = torch.ones((32, 1))
    fake = torch.zeros((32, 1))
    g_loss_total = 0
    d_loss_total = 0
    it = 0
    for real_img, y in dl:
        # print(real_img.shape)
        out = d(real_img)
        real_loss = d_criterion(out, real)
        fake_img = g(torch.randn((32, 512, 1, 1)))
        out = d(fake_img)
        fake_loss = d_criterion(out, fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        
        d_optim.zero_grad()
        d_loss.backward()
        d_optim.step()
        
        fake_img = g(torch.randn((32, 512, 1, 1)))
        out = d(fake_img)
        g_loss = d_criterion(out, real)
        
        g_optim.zero_grad()
        g_loss.backward()
        g_optim.step()
        
        it += 1
        g_loss_total += g
        if it % 10 == 0:
            print(f"G_LOSS: {g_loss_total / it:.4f} - D_LOSS: {d_loss_total / it:.4f}")
        

In [416]:
train_epoch()

Files already downloaded and verified
G_LOSS: 55.5404 - D_LOSS: 0.0000
G_LOSS: 27.3496 - D_LOSS: 105.4875
G_LOSS: 378.8989 - D_LOSS: 72.6059
G_LOSS: 469.4788 - D_LOSS: 111.0616
G_LOSS: 303.8023 - D_LOSS: 96.6940
G_LOSS: 90.5164 - D_LOSS: 48.6301
G_LOSS: 0.0000 - D_LOSS: 3.4300
G_LOSS: 0.0000 - D_LOSS: 44.2696
G_LOSS: 0.0000 - D_LOSS: 48.1975
G_LOSS: 0.4968 - D_LOSS: 35.3923
G_LOSS: 32.7097 - D_LOSS: 13.1348
G_LOSS: 80.8682 - D_LOSS: 0.0001


KeyboardInterrupt: 